In [28]:
import torch 
import torch.nn as nn
from torch.optim import SGD
import numpy as np
import pandas as pd
from tqdm import tqdm # show progress bar
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [39]:
X, y = df.drop("target", axis=1), df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(X)
print(scaler.data_max_)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)
X_train[0]

[ 77.    1.    3.  200.  564.    1.    2.  202.    1.    6.2   2.    4.
   3. ]


array([0.83333333, 1.        , 0.66666667, 0.43396226, 0.29223744,
       0.        , 0.        , 0.57251908, 0.        , 0.32258065,
       0.5       , 0.75      , 1.        ])

In [40]:
# convert numpy array(X) and dataframe.series(y) to pytorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

In [124]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, output_size=2, hidden_size=6, return_logits=True):
        super().__init__()
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc_last = nn.Linear(hidden_size, output_size)
        self.activition = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.return_logits = return_logits
        
    def forward(self, x):
        x = self.fc0(x)
        x = self.activition(x)
        x = self.fc1(x)
        x = self.activition(x)
        x = self.fc2(x)
        x = self.activition(x)
        logits = self.fc_last(x)
        # if using cross-entropy loss -> output should be logits without softmax
        if self.return_logits:
            return logits
        else:
            return self.softmax(logits)        

In [125]:
# make an instance using our defined neural network above, input_size is the number of features from X
# since we are outputing the prob. density function from softmax to be able cope with cross-entropy
model = MLPClassifier(input_size=X_train.shape[1], return_logits=False)

In [126]:
# using AdamW for back-propagation
lr = 0.01 # learning rate
n_epochs = 5000 # how many loops for training the data
optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [127]:
for epoch in range(n_epochs):
    optimizer.zero_grad() # reset gradient storage for the optimizer, has to be done in before every BP
    outputs = model(X_train) # passing the X_train through the whole network and get output
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    # print stats on the fly
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, loss:{loss.item()}")
    

Epoch: 0, loss:0.6933416724205017
Epoch: 100, loss:0.6903453469276428
Epoch: 200, loss:0.6871026754379272
Epoch: 300, loss:0.6772035360336304
Epoch: 400, loss:0.6357824206352234
Epoch: 500, loss:0.5421274900436401
Epoch: 600, loss:0.5011302828788757
Epoch: 700, loss:0.4828343391418457
Epoch: 800, loss:0.4739544987678528
Epoch: 900, loss:0.46825891733169556
Epoch: 1000, loss:0.4618854522705078
Epoch: 1100, loss:0.45619797706604004
Epoch: 1200, loss:0.44991886615753174
Epoch: 1300, loss:0.44423019886016846
Epoch: 1400, loss:0.4392507076263428
Epoch: 1500, loss:0.43463385105133057
Epoch: 1600, loss:0.43037664890289307
Epoch: 1700, loss:0.42676863074302673
Epoch: 1800, loss:0.42382940649986267
Epoch: 1900, loss:0.4215012788772583
Epoch: 2000, loss:0.4194957911968231
Epoch: 2100, loss:0.4178418219089508
Epoch: 2200, loss:0.4164595901966095
Epoch: 2300, loss:0.4152314066886902
Epoch: 2400, loss:0.41411730647087097
Epoch: 2500, loss:0.4130886197090149
Epoch: 2600, loss:0.41216880083084106
Epo